In [1]:
import os
import json
import math
import pandas as pd
from datetime import date, time
import numpy as np
import psycopg2

In [2]:
# Establecer la semilla aleatoria para reproducibilidad
np.random.seed(123)

In [3]:
data = pd.read_csv('../datasets/processed/yellow_analytics/yellow_analytics_part_1.csv', sep=',', encoding='utf8')

In [4]:
data.head()

,IdProveedor,TotalPasajeros,DistanciaViaje,IdTipoTarifa,IdZonaOrigen,IdZonaDestino,IdTipoPago,CostoTotal,FechaRecogida,HoraRecogida,DuracionViaje
0,1,2,3.80,1.0,142,236,1,21.95,2022-01-01,0,1069
1,1,1,2.10,1.0,236,42,1,13.30,2022-01-01,0,504
2,2,1,1.09,1.0,114,68,2,11.80,2022-01-01,0,602
3,2,1,4.30,1.0,68,163,1,30.30,2022-01-01,0,2252
4,2,1,5.07,1.0,233,87,1,26.00,2022-01-01,0,848


In [5]:
data.shape

(364131, 11)

In [6]:
data.IdProveedor.unique()

array([1, 2], dtype=int64)

In [7]:
data.IdProveedor.value_counts()

IdProveedor
2    256555
1    107576
Name: count, dtype: int64

In [8]:
#Calcular el tamaño de muestra proporcional para cada proveedor
proporciones = data['IdProveedor'].value_counts(normalize=True) * 0.12  # Proporción deseada
tamaño_muestra_por_proveedor = (proporciones * len(data)).astype(int)

# Lista para almacenar las muestras aleatorias estratificadas
muestras_estratificadas = []

# Iterar sobre cada proveedor único
for proveedor in data['IdProveedor'].unique():
    # Seleccionar aleatoriamente el tamaño de muestra proporcional para el proveedor actual
    muestra_proveedor = data[data['IdProveedor'] == proveedor].sample(tamaño_muestra_por_proveedor[proveedor])
    # Agregar las muestras seleccionadas al DataFrame de muestras estratificadas
    muestras_estratificadas.append(muestra_proveedor)

# Concatenar las muestras estratificadas en un solo DataFrame
muestras_estratificadas = pd.concat(muestras_estratificadas)

In [9]:
muestras_estratificadas.shape

(43695, 11)

In [10]:
muestras_estratificadas.IdProveedor.value_counts()

IdProveedor
2    30786
1    12909
Name: count, dtype: int64

In [11]:
muestras_estratificadas.head()

,IdProveedor,TotalPasajeros,DistanciaViaje,IdTipoTarifa,IdZonaOrigen,IdZonaDestino,IdTipoPago,CostoTotal,FechaRecogida,HoraRecogida,DuracionViaje
157178,1,1,2.1,1.0,238,48,1,14.55,2022-01-05,8,694
166028,1,1,2.5,1.0,68,229,1,17.75,2022-01-05,12,928
262756,1,1,1.0,1.0,107,164,1,12.35,2022-01-07,16,335
317951,1,3,2.3,1.0,161,246,1,14.80,2022-01-08,18,933
289658,1,2,5.5,1.0,114,238,1,27.35,2022-01-08,2,1247


In [12]:
muestras_estratificadas.dtypes


IdProveedor         int64
TotalPasajeros      int64
DistanciaViaje    float64
IdTipoTarifa      float64
IdZonaOrigen        int64
IdZonaDestino       int64
IdTipoPago          int64
CostoTotal        float64
FechaRecogida      object
HoraRecogida        int64
DuracionViaje       int64
dtype: object

In [13]:
# Conectar a la base de datos PostgreSQL
conn = psycopg2.connect(
    dbname="nycdb",
    user="postgres",
    password="admin",
    host="localhost"
)

In [14]:
# Crear un cursor para ejecutar comandos SQL
cur = conn.cursor()

In [15]:
muestras_estratificadas.columns

Index(['IdProveedor', 'TotalPasajeros', 'DistanciaViaje', 'IdTipoTarifa',
       'IdZonaOrigen', 'IdZonaDestino', 'IdTipoPago', 'CostoTotal',
       'FechaRecogida', 'HoraRecogida', 'DuracionViaje'],
      dtype='object')

In [16]:
muestras_estratificadas.dtypes

IdProveedor         int64
TotalPasajeros      int64
DistanciaViaje    float64
IdTipoTarifa      float64
IdZonaOrigen        int64
IdZonaDestino       int64
IdTipoPago          int64
CostoTotal        float64
FechaRecogida      object
HoraRecogida        int64
DuracionViaje       int64
dtype: object

In [17]:
# Crear la tabla en la base de datos
cur.execute("""
    CREATE TABLE service_yellow (
        Id SERIAL PRIMARY KEY,
        IdProveedor INTEGER,
        TotalPasajeros INTEGER,
        DistanciaViaje FLOAT,
        IdTipoTarifa FLOAT,
        IdZonaOrigen INTEGER,
        IdZonaDestino INTEGER,
        IdTipoPago INTEGER,
        CostoTotal FLOAT,
        FechaRecogida DATE,
        HoraRecogida INTEGER,
        DuracionViaje INTEGER
    )
""")

# Guardar los cambios en la base de datos
conn.commit()


# Convertir los valores del DataFrame a listas de Python
registros = muestras_estratificadas.values.tolist()

# Insertar los registros en la tabla
cur.executemany("""
    INSERT INTO service_yellow (
        IdProveedor, TotalPasajeros, DistanciaViaje, IdTipoTarifa, IdZonaOrigen, IdZonaDestino, IdTipoPago, CostoTotal, FechaRecogida, HoraRecogida, DuracionViaje) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
""", registros)

# Guardar los cambios en la base de datos
conn.commit()

# Cerrar la conexión con la base de datos
cur.close()
conn.close()

print("Registros migrados exitosamente a PostgreSQL.")

Registros migrados exitosamente a PostgreSQL.
